In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

In [2]:
class LSTM(nn.Module):
  def __init__(self,input_size, hidden_size, output_size, num_layers):
    super(LSTM,self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                        num_layers = num_layers, batch_first = True, bias = True)
    self.relu = nn.ReLU()
    self.fc = nn.Linear(hidden_dim, output_size)

  def forward(self,x):
    # num_layers * batch_size * hidden_dim
    # if batch_size == 1
    # hidden_dim크기의 1차원 벡터가 num_layers의 갯수만큼 존재
    h_0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
    c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

    # hidden layer에 대한 선형 회귀
    output ,(hn,cn) = self.lstm(x,(h_0, c_0))
    out = self.relu(hn)
    out = self.fc(out)
    return out

In [3]:
class new_LSTM(nn.Module):
  def __init__(self,input_size, hidden_size, output_size, num_layers):
    super(LSTM,self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                        num_layers = num_layers, batch_first = True, bias = True)
    self.softmax = nn.Softmax(dim = 0)
    self.fc = nn.Linear(hidden_dim, output_size)

  def forward(self,x):
    # num_layers * batch_size * hidden_dim
    # if batch_size == 1
    # hidden_dim크기의 1차원 벡터가 num_layers의 갯수만큼 존재
    h_0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
    c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

    # hidden layer에 대한 선형 회귀
    output ,(hn,cn) = self.lstm(x,(h_0, c_0))
    out = self.softmax(hn)
    out = self.fc(out)
    return out

In [4]:
df = pd.read_csv('./train.csv')
df.head()

FileNotFoundError: ignored

In [ ]:
df = df.drop(columns = 'Unnamed: 0')

In [ ]:
df.head()

,num
0,39.7
1,41.7
2,46.4
3,54.9
4,60.7


In [ ]:
data_np = df.values
data_dif = []
for i in range(len(data_np) - 1):
  data_dif.append(data_np[i+1] - data_np[i])

In [ ]:
dif_max = max(data_dif)
print(dif_max)

[129.7]


In [ ]:
# 기존 lstm
# input = 1, hidden = 32, output = 1, layers = 1
model = LSTM(1, 32 , 1, 1)
critertion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
num_epochs = 1000
window_size = 5

In [ ]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

train_x = []
train_y = []
for i in range(len(data_np) - window_size - 1):
  train_x.append(data_np[i:i+window_size])
  train_y.append(data_np[i+window_size])
print(f'train_x shape : {train_x.shape}, train_y shape : {train_y.shape}')

train_x = torch.FloatTensor(train_x)
train_y = torch.FloatTensor(train_y)

dataset = TensorDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
# train

for epoch in range(num_epochs + 1):
  for X,y in dataloader:
    y_pred = model(X)
    loss = critertion(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if epoch % 100 == 0:
    print(f'Epoch : {epoch} , loss : {loss.item():.2f}')